In [10]:
import numpy as np
#from pydgrid.pydgrid import grid
from pydgrid.plot_bokeh import plot_results
import sympy as sym
import pydae.build as db
from pydae.grid_urisi import unb_ri_si
import json

In [11]:
data = {
        "buses":[
                 {"bus": "B1",  "pos_x":   0, "pos_y":  0, "units": "m", "U_kV":0.4},
                 {"bus": "B2",  "pos_x":  20, "pos_y":  0, "units": "m", "U_kV":0.4},
                 {"bus": "B3",  "pos_x": 120, "pos_y":  0, "units": "m", "U_kV":0.4},
                 {"bus": "B4",  "pos_x": 140, "pos_y":  0, "units": "m", "U_kV":0.4}
                ],
        "grid_formers":[
                      #  {"bus": "B1",
                      #  "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
                      #  "kV": [0.231, 0.231, 0.231]},
                      # {"bus": "B4",
                      # "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
                     #  "kV": [0.231, 0.231, 0.231]}
                       ],
        "lines":[
                 {"bus_j": "B1",  "bus_k": "B2",  "code": "lv_cu_150", "m":  20.0,'monitor':True},
                 {"bus_j": "B2",  "bus_k": "B3",  "code": "lv_cu_150", "m": 100.0,'monitor':True},
                 {"bus_j": "B3",  "bus_k": "B4",  "code": "lv_cu_150", "m":  20.0,'monitor':True}
                ],
        "loads":[
                 {"bus": "B2" , "kVA": [0.1]*3, "pf":[ 1]*3,"type":"3P+N"},
                 {"bus": "B3" , "kVA": [0.1]*3, "pf":[ 1]*3,"type":"3P+N"},
                # {"bus": "B4" , "kVA": [0.1]*3, "pf":[ 1]*3,"type":"3P+N"},
                ],
        "shunts":[
                 #{"bus": "B1" , "R": 0.001, "X": 0.0, "bus_nodes": [4,0]},
                 #{"bus": "B4" , "R": 0.001, "X": 0.0, "bus_nodes": [4,0]}
                 ],
        "line_codes":
            {"lv_cu_150":  {"Rph":0.167,"Xph":0.08, "Rn":0.167, "Xn": 0.08}
            }
       }

In [12]:
alpha = np.exp(2.0/3*np.pi*1j)
A_0a =  np.array([[1, 1, 1],
                  [1, alpha**2, alpha],
                  [1, alpha, alpha**2]])

A_a0 = 1/3* np.array([[1, 1, 1],
                      [1, alpha, alpha**2],
                      [1, alpha**2, alpha]])

In [15]:
grid_4bus = unb_ri_si(data)

params_dict  = grid_4bus.dae['params']
f_list = grid_4bus.dae['f']
x_list = grid_4bus.dae['x']
g_list = grid_4bus.dae['g'] 
y_list = grid_4bus.dae['y'] 
u_dict = grid_4bus.dae['u']
h_dict = grid_4bus.dae['h_v_m_dict']

omega_coi_i = 0
HS_coi = 0

omega_coi = sym.Symbol('omega_coi',real=True)
xi_freq = sym.Symbol('xi_freq',real=True)
K_agc = sym.Symbol('K_agc',real=True)

vscs = [
       {'bus':'B1','S_n':100e3,'R':0.01,'X':0.1,'K_f':0.1,'T_f':1.0,'K_sec':0.5,'K_delta':0.001},
       {'bus':'B4','S_n':100e3,'R':0.01,'X':0.1,'K_f':0.1,'T_f':1.0,'K_sec':0.5,'K_delta':0.000},
       ]
for vsc in vscs:
    
    name = vsc['bus']

    # inputs
    e_an,e_bn,e_cn,e_ng = sym.symbols(f'e_{name}_an,e_{name}_bn,e_{name}_cn,e_{name}_ng', real=True)
    omega_ref,p_ref = sym.symbols(f'omega_{name}_ref,p_{name}_ref', real=True)
    
    # parameters
    S_n,H,K_f,T_f,K_sec,K_delta  = sym.symbols(f'S_n_{name},H_{name},K_f_{name},T_f_{name},K_sec_{name},K_delta_{name}', real=True)
    R_sa,R_sb,R_sc,R_sn,R_ng = sym.symbols(f'R_{name}_sa,R_{name}_sb,R_{name}_sc,R_{name}_sn,R_{name}_ng', real=True)
    X_sa,X_sb,X_sc,X_sn,X_ng = sym.symbols(f'X_{name}_sa,X_{name}_sb,X_{name}_sc,X_{name}_sn,X_{name}_ng', real=True)
    
    # dynamical states
    phi = sym.Symbol(f'phi_{name}', real=True)
    omega = sym.Symbol(f'omega_{name}', real=True)
    
    # algebraic states
    #e_an_i,e_bn_i,e_cn_i,e_ng_i = sym.symbols(f'e_{name}_an_i,e_{name}_bn_i,e_{name}_cn_i,e_{name}_ng_i', real=True)
    v_sa_r,v_sb_r,v_sc_r,v_sn_r,v_ng_r = sym.symbols(f'v_{name}_a_r,v_{name}_b_r,v_{name}_c_r,v_{name}_n_r,v_{name}_n_r', real=True)
    v_sa_i,v_sb_i,v_sc_i,v_sn_i,v_ng_i = sym.symbols(f'v_{name}_a_i,v_{name}_b_i,v_{name}_c_i,v_{name}_n_i,v_{name}_n_i', real=True)
    i_sa_r,i_sb_r,i_sc_r,i_sn_r,i_ng_r = sym.symbols(f'i_{name}_a_r,i_{name}_b_r,i_{name}_c_r,i_{name}_n_r,i_{name}_ng_r', real=True)
    i_sa_i,i_sb_i,i_sc_i,i_sn_i,i_ng_i = sym.symbols(f'i_{name}_a_i,i_{name}_b_i,i_{name}_c_i,i_{name}_n_i,i_{name}_ng_i', real=True)
    omega = sym.Symbol(f'omega_{name}', real=True)
    
    e_ng_r,e_ng_i = sym.symbols(f'e_{name}_ng_r,e_{name}_ng_i', real=True)
    
    Z_sa = R_sa + 1j*X_sa
    Z_sb = R_sb + 1j*X_sb
    Z_sc = R_sc + 1j*X_sc
    Z_sn = R_sn + 1j*X_sn
    Z_ng = R_ng + 1j*X_ng

    i_sa = i_sa_r + 1j*i_sa_i
    i_sb = i_sb_r + 1j*i_sb_i
    i_sc = i_sc_r + 1j*i_sc_i
    i_sn = i_sn_r + 1j*i_sn_i
    i_ng = i_ng_r + 1j*i_ng_i

    v_sa = v_sa_r + 1j*v_sa_i
    v_sb = v_sb_r + 1j*v_sb_i
    v_sc = v_sc_r + 1j*v_sc_i
    v_sn = v_sn_r + 1j*v_sn_i
    v_ng = v_ng_r + 1j*v_ng_i
    
    e_an_r = e_an*sym.cos(phi) 
    e_an_i = e_an*sym.sin(phi) 
    e_bn_r = e_bn*sym.cos(phi-2/3*np.pi) 
    e_bn_i = e_bn*sym.sin(phi-2/3*np.pi) 
    e_cn_r = e_cn*sym.cos(phi-4/3*np.pi) 
    e_cn_i = e_cn*sym.sin(phi-4/3*np.pi) 
    
    e_an_cplx = e_an_r + 1j*e_an_i
    e_bn_cplx = e_bn_r + 1j*e_bn_i
    e_cn_cplx = e_cn_r + 1j*e_cn_i
    e_ng_cplx = e_ng_r + 1j*e_ng_i

    v_san = v_sa - v_sn
    v_sbn = v_sb - v_sn
    v_scn = v_sc - v_sn

    eq_i_sa_cplx = e_an_cplx - i_sa*Z_sa - v_san
    eq_i_sb_cplx = e_bn_cplx - i_sb*Z_sb - v_sbn
    eq_i_sc_cplx = e_cn_cplx - i_sc*Z_sc - v_scn
    eq_i_sn_cplx = e_ng_cplx - i_sn*Z_sn - v_ng
    eq_i_ng_cplx = -i_ng + i_sa + i_sb + i_sc + i_sn
    eq_e_ng_cplx  = -e_ng_cplx  + i_ng*Z_ng

    g_list += [sym.re(eq_i_sa_cplx)] 
    g_list += [sym.re(eq_i_sb_cplx)] 
    g_list += [sym.re(eq_i_sc_cplx)] 
    g_list += [sym.re(eq_i_sn_cplx)] 
    g_list += [sym.re(eq_i_ng_cplx)] 
    g_list += [sym.re(eq_e_ng_cplx)] 
    g_list += [sym.im(eq_i_sa_cplx)] 
    g_list += [sym.im(eq_i_sb_cplx)] 
    g_list += [sym.im(eq_i_sc_cplx)] 
    g_list += [sym.im(eq_i_sn_cplx)] 
    g_list += [sym.im(eq_i_ng_cplx)] 
    g_list += [sym.im(eq_e_ng_cplx)]


    y_list += [i_sa_r,i_sb_r,i_sc_r,i_sn_r,i_ng_r,e_ng_r]
    y_list += [i_sa_i,i_sb_i,i_sc_i,i_sn_i,i_ng_i,e_ng_i]

    y_ini_str = [str(item) for item in y_list]

    for ph in ['a','b','c','n']:
        i_s_r = sym.Symbol(f'i_{name}_{ph}_r', real=True)
        i_s_i = sym.Symbol(f'i_{name}_{ph}_i', real=True)                   
        g_list[y_ini_str.index(f'v_{name}_{ph}_r')] += i_s_r
        g_list[y_ini_str.index(f'v_{name}_{ph}_i')] += i_s_i
        


    V_1 = 400/np.sqrt(3)
#    V_1 = 400/np.sqrt(3)*np.exp(1j*np.deg2rad(0))
   # A_1toabc = np.array([1, alpha**2, alpha])
    #V_abc = V_1 * A_1toabc 
    #e_an_r,e_bn_r,e_cn_r = V_abc.real
    #e_an_i,e_bn_i,e_cn_i = V_abc.imag

    u_dict.update({f'e_{name}_an':V_1,f'e_{name}_bn':V_1,f'e_{name}_cn':V_1})
    u_dict.update({f'phi_{name}':0.0})
    u_dict.update({f'p_{name}_ref':0.0})
    u_dict.update({f'omega_{name}_ref':1.0})

    for ph in ['a','b','c','n']:
        u_dict.pop(f'i_{name}_{ph}_r')
        u_dict.pop(f'i_{name}_{ph}_i')

    for ph in ['a','b','c','n']:
        params_dict.update({f'X_{name}_s{ph}':vsc['X'],f'R_{name}_s{ph}':vsc['R']})
        
    params_dict.update({f'S_n_{name}':vsc['S_n']})
    params_dict.update({f'X_{name}_ng':vsc['X'],f'R_{name}_ng':vsc['R']})
    params_dict.update({f'K_f_{name}':vsc['K_f']})
    params_dict.update({f'T_f_{name}':vsc['T_f']})
    params_dict.update({f'K_sec_{name}':vsc['K_sec']})
    params_dict.update({f'K_delta_{name}':vsc['K_delta']})
    
    
    v_sabc = sym.Matrix([[v_sa],[v_sb],[v_sc]])
    i_sabc = sym.Matrix([[i_sa],[i_sb],[i_sc]])
    
    v_szpn = A_a0*v_sabc
    i_szpn = A_a0*i_sabc
    
    s_pos = 3*v_szpn[1]*sym.conjugate(i_szpn[1])
    s_neg = 3*v_szpn[2]*sym.conjugate(i_szpn[2])
    s_zer = 3*v_szpn[0]*sym.conjugate(i_szpn[0])
    
    p_pos = sym.re(s_pos)
    
    dphi   = 2*np.pi*50*(omega - omega_coi) - K_delta*phi
    domega = 1/T_f*(omega_ref - K_f*(p_ref + K_sec*xi_freq - p_pos)/S_n - omega)
    
    f_list += [dphi,domega]
    x_list += [ phi, omega]
    
    h_dict.update({f'p_{name}_pos':sym.re(s_pos),f'p_{name}_neg':sym.re(s_neg),f'p_{name}_zer':sym.re(s_zer)})
    h_dict.update({str(e_an):e_an,str(e_bn):e_bn,str(e_cn):e_cn})
    h_dict.update({str(p_ref):p_ref,str(omega_ref):omega_ref})
    HS_coi += S_n
    omega_coi_i+= S_n*omega



g_list += [omega_coi -  omega_coi_i/HS_coi]
y_list += [omega_coi]


dxi_freq = K_agc*(1 - omega_coi)

f_list += [dxi_freq]
x_list += [ xi_freq]


params_dict.update({f'K_agc':0.001})



Matrix(16, 0, [])


In [16]:
sys_dict = {'name':'oc_4bus4wire2src',
           'params_dict':params_dict,
           'f_list':f_list,
           'g_list':g_list,
           'x_list':x_list,
           'y_ini_list':y_list,
           'y_run_list':y_list,
           'u_run_dict':u_dict,
           'u_ini_dict':u_dict,
           'h_dict':h_dict
           }

db.system(sys_dict)
db.sys2num(sys_dict)

data_new = json.dumps(grid_4bus.dae['xy_0_dict'], indent=4)
fobj = open("xy_0_dict.json","w")
fobj.write(data_new)
fobj.close()

In [83]:
grid_4bus.pf()  # solve power flow
p=plot_results(grid_4bus)

In [46]:
Y_ii = grid_4bus.Y_ii.toarray()
Y_vv = grid_4bus.Y_vv
Y_vi = grid_4bus.Y_vi
inv_Y_ii = np.linalg.inv(Y_ii)
N_nz_nodes = grid_4bus.params_pf[0].N_nz_nodes
N_v = grid_4bus.params_pf[0].N_nodes_v
nodes_list = grid_4bus.nodes
Y_primitive = grid_4bus.Y_primitive_sp.toarray() 
A_conect = grid_4bus.A_sp.toarray()
node_sorter  = grid_4bus.node_sorter
N_v = grid_4bus.N_nodes_v

np.savez('matrices',Y_primitive=Y_primitive,A_conect=A_conect,nodes_list=nodes_list,
         node_sorter=node_sorter,N_v=N_v, Y_vv=Y_vv, Y_vi=Y_vi)


with open("grid_data.json", "w") as fobj:
    json.dump(grid_4bus.data, fobj, indent=4, sort_keys=True)


In [86]:
print(sys_dict['g'][y_ini_str.index('v_B1_n_i')])

i_B1_n_i - 1243.51832949342*v_B1_n_i + 116.655487182478*v_B1_n_r + 243.518329493424*v_B2_n_i - 116.655487182478*v_B2_n_r


In [115]:
sys_dict['y_ini'].T

Matrix([
[     v_B2_a_r],
[     v_B2_a_i],
[     v_B2_b_r],
[     v_B2_b_i],
[     v_B2_c_r],
[     v_B2_c_i],
[     v_B2_n_r],
[     v_B2_n_i],
[     v_B3_a_r],
[     v_B3_a_i],
[     v_B3_b_r],
[     v_B3_b_i],
[     v_B3_c_r],
[     v_B3_c_i],
[     v_B3_n_r],
[     v_B3_n_i],
[     v_B1_a_r],
[     v_B1_a_i],
[     v_B1_b_r],
[     v_B1_b_i],
[     v_B1_c_r],
[     v_B1_c_i],
[     v_B1_n_r],
[     v_B1_n_i],
[     v_B4_a_r],
[     v_B4_a_i],
[     v_B4_b_r],
[     v_B4_b_i],
[     v_B4_c_r],
[     v_B4_c_i],
[     v_B4_n_r],
[     v_B4_n_i],
[i_l_B1_B2_a_r],
[i_l_B1_B2_a_i],
[i_l_B1_B2_b_r],
[i_l_B1_B2_b_i],
[i_l_B1_B2_c_r],
[i_l_B1_B2_c_i],
[i_l_B1_B2_n_r],
[i_l_B1_B2_n_i],
[i_l_B2_B3_a_r],
[i_l_B2_B3_a_i],
[i_l_B2_B3_b_r],
[i_l_B2_B3_b_i],
[i_l_B2_B3_c_r],
[i_l_B2_B3_c_i],
[i_l_B2_B3_n_r],
[i_l_B2_B3_n_i],
[i_l_B3_B4_a_r],
[i_l_B3_B4_a_i],
[i_l_B3_B4_b_r],
[i_l_B3_B4_b_i],
[i_l_B3_B4_c_r],
[i_l_B3_B4_c_i],
[i_l_B3_B4_n_r],
[i_l_B3_B4_n_i],
[i_load_B2_a_r],
[i_load_B2_a_i],
[i_lo